In [1]:
import numpy as np
from keras import layers
from keras import optimizers
from keras.layers import Input, Add, Dense, Activation, Dropout, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.models import Model
from keras.initializers import glorot_uniform
import os
from matplotlib import image
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
def one_hot_encoder(labellist, nb_classes):
    one_hot = np.zeros([len(labellist), nb_classes])
    for i,l in enumerate(labellist):
        one_hot[i][int(l)-1] = 1
    return one_hot

In [13]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = Conv2D(filters = F1, kernel_size=(1,1), strides=(1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters = F2, kernel_size=(f,f), strides=(1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters = F3, kernel_size=(1,1), strides=(1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
    
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    X = Dropout(rate=0.2)(X)
    
    return X

In [14]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = Conv2D(F1, (1,1), strides = (s,s), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(F2, (f,f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(F3, (1,1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name = bn_name_base + '2c')(X)
    
    X_shortcut = Conv2D(F3, (1,1), strides=(s,s), padding='valid', name = conv_name_base+ '1', kernel_initializer= glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)
    
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    X = Dropout(rate=0.2)(X)
    
    return X

In [15]:
def ResNet50(input_shape= (64,64,3), classes=6):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    X_input = Input(input_shape)
    
    X = ZeroPadding2D((3,3))(X_input)
    
    #Stage 1
    X = Conv2D(64, (7,7), strides = (2,2), name = 'conv1', kernel_initializer= glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3,3), strides=(2,2))(X)
    #X = Dropout(rate=0.2)(X)
    
    #Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64,64, 256], stage = 2, block = 'b')
    X = identity_block(X, 3, [64,64, 256], stage = 2, block = 'c')
    
    #Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage = 3, block = 'b')
    X = identity_block(X, 3, [128, 128, 512], stage = 3, block = 'c')
    X = identity_block(X, 3, [128, 128, 512], stage = 3, block = 'd')
    
    #Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage = 4, block = 'b')
    X = identity_block(X, 3, [256, 256, 1024], stage = 4, block = 'c')
    X = identity_block(X, 3, [256, 256, 1024], stage = 4, block = 'd')
    X = identity_block(X, 3, [256, 256, 1024], stage = 4, block = 'e')
    X = identity_block(X, 3, [256, 256, 1024], stage = 4, block = 'f')
        
    #Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage = 5, block = 'b')
    X = identity_block(X, 3, [512, 512, 2048], stage = 5, block = 'c')
    
    X = AveragePooling2D(pool_size=(2,2), strides = None, padding='valid', data_format=None)(X)
    
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer= glorot_uniform(seed=0))(X)
    
    model = Model(inputs = X_input, outputs = X, name='ResNets50')
    
    return model

In [6]:
ftraindir = r'C:\Users\dhecker\Downloads\dice\train'
ftestdir = r'C:\Users\dhecker\Downloads\dice\valid'
IMSIZE = 64

In [7]:
#load training data
dirs = ['d6','d8','d10','d12','d20']
loaded_images = list()
labels = []
for d in dirs:
    files = os.listdir(os.path.join(ftraindir,d))
    for f in files:
        im = image.imread(os.path.join(ftraindir,d,f))
        im = np.resize(im, (IMSIZE,IMSIZE,3))
        loaded_images.append(im) #images as LIST
        
        #print('> loaded %s %s' % (f, im.shape))
        
        #set label
        if f[0:2] == 'd4':
            labels.append(1)
        elif f[0:2] == 'd6':
            labels.append(2)
        elif f[0:2] == 'd8':
            labels.append(3)
        elif f[0:3] == 'd10':
            labels.append(4)
        elif f[0:3] == 'd12':
            labels.append(5)
        elif f[0:3] == 'd20':
            labels.append(6)
            
X_train_orig = np.array(loaded_images)
Y_train_orig = labels
print('loaded %i images and set %i labels' % (X_train_orig.shape[0], len(labels)))

loaded 12337 images and set 12337 labels


In [8]:
#load test data
loaded_images = list()
labels = []
for d in dirs:
    files = os.listdir(os.path.join(ftestdir,d))
    for f in files:
        if f[0] == 'd':
            
            im = image.imread(os.path.join(ftestdir,d,f))
            im = np.resize(im, (IMSIZE,IMSIZE,3))
            loaded_images.append(im)
        
            #set label
            if f[0:2] == 'd4':
                labels.append(1)
            elif f[0:2] == 'd6':
                labels.append(2)
            elif f[0:2] == 'd8':
                labels.append(3)
            elif f[0:3] == 'd10':
                labels.append(4)
            elif f[0:3] == 'd12':
                labels.append(5)
            elif f[0:3] == 'd20':
                labels.append(6)
            
X_test_orig = np.array(loaded_images)
Y_test_orig = labels
print('loaded %i images and set %i labels' % (X_test_orig.shape[0], len(Y_test_orig)))

loaded 1761 images and set 1761 labels


In [9]:
X_train = X_train_orig/255.
X_test = X_test_orig/255.
Y_train = one_hot_encoder(Y_train_orig, 6)
Y_test = one_hot_encoder(Y_test_orig, 6)
print("number of training examples = " + str(X_train.shape[0]))
print("number of test examples = " + str(X_test.shape[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape)) 
print("X_test.shape: " + str(X_test.shape))
print("Y_test.shape: " + str(Y_test.shape))
conv_layers = {}

number of training examples = 12337
number of test examples = 1761
X_train shape: (12337, 64, 64, 3)
Y_train shape: (12337, 6)
X_test.shape: (1761, 64, 64, 3)
Y_test.shape: (1761, 6)


In [18]:
#20% dropout at the end of every ident and conv block
model = ResNet50(input_shape=(IMSIZE,IMSIZE,3), classes = 6)
adam = optimizers.Adam(0.00001)#, 0.9, 0.999, False)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=250, batch_size=32)
preds = model.evaluate(X_test, Y_test)
print('Loss = ' + str(preds[0]))
print('Test Accuracy = ' + str(preds[1]))

Epoch 1/250
12337/12337 [==============================] - 125s 10ms/step - loss: 1.9483 - acc: 0.2856
Epoch 2/250
12337/12337 [==============================] - 107s 9ms/step - loss: 1.6798 - acc: 0.3304
Epoch 3/250
12337/12337 [==============================] - 109s 9ms/step - loss: 1.5771 - acc: 0.3375
Epoch 4/250
12337/12337 [==============================] - 107s 9ms/step - loss: 1.5271 - acc: 0.3552
Epoch 5/250
12337/12337 [==============================] - 108s 9ms/step - loss: 1.4905 - acc: 0.3584
Epoch 6/250
12337/12337 [==============================] - 109s 9ms/step - loss: 1.4440 - acc: 0.3691
Epoch 7/250
12337/12337 [==============================] - 107s 9ms/step - loss: 1.4289 - acc: 0.3744
Epoch 8/250
12337/12337 [==============================] - 384s 31ms/step - loss: 1.4050 - acc: 0.3828
Epoch 9/250
12337/12337 [==============================] - 352s 29ms/step - loss: 1.4145 - acc: 0.3782
Epoch 10/250
12337/12337 [==============================] - 351s 28ms/step - lo

12337/12337 [==============================] - 354s 29ms/step - loss: 0.4441 - acc: 0.8119
Epoch 80/250
12337/12337 [==============================] - 327s 26ms/step - loss: 0.4365 - acc: 0.8173
Epoch 81/250
12337/12337 [==============================] - 312s 25ms/step - loss: 0.4214 - acc: 0.8231
Epoch 82/250
12337/12337 [==============================] - 310s 25ms/step - loss: 0.4152 - acc: 0.8289
Epoch 83/250
12337/12337 [==============================] - 311s 25ms/step - loss: 0.4171 - acc: 0.8241
Epoch 84/250
12337/12337 [==============================] - 324s 26ms/step - loss: 0.4096 - acc: 0.8291
Epoch 85/250
12337/12337 [==============================] - 352s 29ms/step - loss: 0.4076 - acc: 0.8296
Epoch 86/250
12337/12337 [==============================] - 352s 29ms/step - loss: 0.4158 - acc: 0.8292
Epoch 87/250
12337/12337 [==============================] - 353s 29ms/step - loss: 0.4011 - acc: 0.8290
Epoch 88/250
12337/12337 [==============================] - 354s 29ms/step - 

12337/12337 [==============================] - 355s 29ms/step - loss: 0.2541 - acc: 0.8923
Epoch 158/250
12337/12337 [==============================] - 355s 29ms/step - loss: 0.2518 - acc: 0.8971
Epoch 159/250
12337/12337 [==============================] - 355s 29ms/step - loss: 0.2540 - acc: 0.8914
Epoch 160/250
12337/12337 [==============================] - 355s 29ms/step - loss: 0.2633 - acc: 0.8902
Epoch 161/250
12337/12337 [==============================] - 357s 29ms/step - loss: 0.2502 - acc: 0.8961
Epoch 162/250
12337/12337 [==============================] - 357s 29ms/step - loss: 0.2404 - acc: 0.8983
Epoch 163/250
12337/12337 [==============================] - 355s 29ms/step - loss: 0.2506 - acc: 0.8950
Epoch 164/250
12337/12337 [==============================] - 356s 29ms/step - loss: 0.2435 - acc: 0.8984
Epoch 165/250
12337/12337 [==============================] - 355s 29ms/step - loss: 0.2436 - acc: 0.8987
Epoch 166/250
12337/12337 [==============================] - 354s 29m

12337/12337 [==============================] - 109s 9ms/step - loss: 0.1952 - acc: 0.9172
Epoch 236/250
12337/12337 [==============================] - 108s 9ms/step - loss: 0.1867 - acc: 0.9213
Epoch 237/250
12337/12337 [==============================] - 108s 9ms/step - loss: 0.1757 - acc: 0.9254
Epoch 238/250
12337/12337 [==============================] - 108s 9ms/step - loss: 0.1818 - acc: 0.9224
Epoch 239/250
12337/12337 [==============================] - 108s 9ms/step - loss: 0.1737 - acc: 0.9276
Epoch 240/250
12337/12337 [==============================] - 109s 9ms/step - loss: 0.1804 - acc: 0.9241
Epoch 241/250
12337/12337 [==============================] - 108s 9ms/step - loss: 0.1796 - acc: 0.9221
Epoch 242/250
12337/12337 [==============================] - 108s 9ms/step - loss: 0.1748 - acc: 0.9249
Epoch 243/250
12337/12337 [==============================] - 108s 9ms/step - loss: 0.1862 - acc: 0.9238
Epoch 244/250
12337/12337 [==============================] - 108s 9ms/step - l

In [17]:
#CONTINUED FROM CELL BELOW!!!
#20% dropout at the end of every ident and conv block
#model = ResNet50(input_shape=(IMSIZE,IMSIZE,3), classes = 6)
#adam = optimizers.Adam(0.0001)#, 0.9, 0.999, False)
#model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=50, batch_size=32)
preds = model.evaluate(X_test, Y_test)
print('Loss = ' + str(preds[0]))
print('Test Accuracy = ' + str(preds[1]))

Epoch 1/50
12337/12337 [==============================] - 109s 9ms/step - loss: 0.2404 - acc: 0.8976
Epoch 2/50
12337/12337 [==============================] - 109s 9ms/step - loss: 0.2507 - acc: 0.8969
Epoch 3/50
12337/12337 [==============================] - 109s 9ms/step - loss: 0.2373 - acc: 0.9031
Epoch 4/50
12337/12337 [==============================] - 108s 9ms/step - loss: 0.2516 - acc: 0.9006
Epoch 5/50
12337/12337 [==============================] - 108s 9ms/step - loss: 0.2418 - acc: 0.8991
Epoch 6/50
12337/12337 [==============================] - 108s 9ms/step - loss: 0.2278 - acc: 0.9072
Epoch 7/50
12337/12337 [==============================] - 108s 9ms/step - loss: 0.2219 - acc: 0.9057
Epoch 8/50
12337/12337 [==============================] - 108s 9ms/step - loss: 0.2180 - acc: 0.9101
Epoch 9/50
12337/12337 [==============================] - 108s 9ms/step - loss: 0.2249 - acc: 0.9081
Epoch 10/50
12337/12337 [==============================] - 108s 9ms/step - loss: 0.2090 - a

In [16]:
#20% dropout at the end of every ident and conv block
#model = ResNet50(input_shape=(IMSIZE,IMSIZE,3), classes = 6)
#adam = optimizers.Adam(0.0001)#, 0.9, 0.999, False)
#model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=50, batch_size=32)
preds = model.evaluate(X_test, Y_test)
print('Loss = ' + str(preds[0]))
print('Test Accuracy = ' + str(preds[1]))

Epoch 1/50
12337/12337 [==============================] - 123s 10ms/step - loss: 1.6230 - acc: 0.3461
Epoch 2/50
12337/12337 [==============================] - 106s 9ms/step - loss: 1.3821 - acc: 0.3976
Epoch 3/50
12337/12337 [==============================] - 107s 9ms/step - loss: 1.2252 - acc: 0.4637
Epoch 4/50
12337/12337 [==============================] - 107s 9ms/step - loss: 1.1350 - acc: 0.4920
Epoch 5/50
12337/12337 [==============================] - 107s 9ms/step - loss: 1.0317 - acc: 0.5297
Epoch 6/50
12337/12337 [==============================] - 107s 9ms/step - loss: 0.9532 - acc: 0.5761
Epoch 7/50
12337/12337 [==============================] - 106s 9ms/step - loss: 0.8805 - acc: 0.6225
Epoch 8/50
12337/12337 [==============================] - 106s 9ms/step - loss: 0.8067 - acc: 0.6703
Epoch 9/50
12337/12337 [==============================] - 106s 9ms/step - loss: 0.7207 - acc: 0.7024
Epoch 10/50
12337/12337 [==============================] - 107s 9ms/step - loss: 0.6774 - 

In [12]:
#dropout after every activation layer in ident and conv blocks
model = ResNet50(input_shape=(IMSIZE,IMSIZE,3), classes = 6)
adam = optimizers.Adam(0.0001)#, 0.9, 0.999, False)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=32)
preds = model.evaluate(X_test, Y_test)
print('Loss = ' + str(preds[0]))
print('Test Accuracy = ' + str(preds[1]))

Epoch 1/10
12337/12337 [==============================] - 125s 10ms/step - loss: 1.6052 - acc: 0.3460
Epoch 2/10
12337/12337 [==============================] - 109s 9ms/step - loss: 1.4151 - acc: 0.4006
Epoch 3/10
12337/12337 [==============================] - 109s 9ms/step - loss: 1.2328 - acc: 0.4641
Epoch 4/10
12337/12337 [==============================] - 109s 9ms/step - loss: 1.1140 - acc: 0.5038
Epoch 5/10
12337/12337 [==============================] - 110s 9ms/step - loss: 0.9923 - acc: 0.5466
Epoch 6/10
12337/12337 [==============================] - 109s 9ms/step - loss: 0.9541 - acc: 0.5715
Epoch 7/10
12337/12337 [==============================] - 109s 9ms/step - loss: 0.8930 - acc: 0.6097
Epoch 8/10
12337/12337 [==============================] - 110s 9ms/step - loss: 0.7892 - acc: 0.6631
Epoch 9/10
12337/12337 [==============================] - 107s 9ms/step - loss: 0.7486 - acc: 0.6834
Epoch 10/10
1761/1761 [==============================] - 6s 3ms/step
Loss = 4.560514301687

In [11]:
model = ResNet50(input_shape=(IMSIZE,IMSIZE,3), classes = 6)
adam = optimizers.Adam(0.0001, 0.8, 0.999, False)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=32)
preds = model.evaluate(X_test, Y_test)
print('Loss = ' + str(preds[0]))
print('Test Accuracy = ' + str(preds[1]))

Epoch 1/10
12337/12337 [==============================] - 107s 9ms/step - loss: 1.7078 - acc: 0.3263
Epoch 2/10
12337/12337 [==============================] - 94s 8ms/step - loss: 1.5943 - acc: 0.3280
Epoch 3/10
12337/12337 [==============================] - 94s 8ms/step - loss: 1.5600 - acc: 0.3280
Epoch 4/10
12337/12337 [==============================] - 93s 8ms/step - loss: 1.5453 - acc: 0.3280
Epoch 5/10
12337/12337 [==============================] - 93s 8ms/step - loss: 1.5381 - acc: 0.3280
Epoch 6/10
12337/12337 [==============================] - 93s 8ms/step - loss: 1.5341 - acc: 0.3280
Epoch 7/10
12337/12337 [==============================] - 93s 8ms/step - loss: 1.5317 - acc: 0.3280
Epoch 8/10
12337/12337 [==============================] - 93s 8ms/step - loss: 1.5307 - acc: 0.3280
Epoch 9/10
12337/12337 [==============================] - 93s 8ms/step - loss: 1.5296 - acc: 0.3280
Epoch 10/10
1761/1761 [==============================] - 5s 3ms/step
Loss = 1.5276660708660839
Test

In [17]:
model = ResNet50(input_shape=(IMSIZE,IMSIZE,3), classes = 6)
adam = optimizers.Adam(0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=32)
preds = model.evaluate(X_test, Y_test)
print('Loss = ' + str(preds[0]))
print('Test Accuracy = ' + str(preds[1]))

Epoch 1/10
12337/12337 [==============================] - 112s 9ms/step - loss: 1.1836 - acc: 0.5345
Epoch 2/10
12337/12337 [==============================] - 96s 8ms/step - loss: 0.8883 - acc: 0.6425
Epoch 3/10
12337/12337 [==============================] - 96s 8ms/step - loss: 0.7298 - acc: 0.6981
Epoch 4/10
12337/12337 [==============================] - 96s 8ms/step - loss: 0.6228 - acc: 0.7335
Epoch 5/10
12337/12337 [==============================] - 96s 8ms/step - loss: 0.5630 - acc: 0.7673
Epoch 6/10
12337/12337 [==============================] - 96s 8ms/step - loss: 0.5274 - acc: 0.7744
Epoch 7/10
12337/12337 [==============================] - 97s 8ms/step - loss: 0.4809 - acc: 0.7945
Epoch 8/10
12337/12337 [==============================] - 97s 8ms/step - loss: 0.4692 - acc: 0.8042
Epoch 9/10
12337/12337 [==============================] - 96s 8ms/step - loss: 0.4664 - acc: 0.8098
Epoch 10/10
1761/1761 [==============================] - 6s 4ms/step
Loss = 1.6051093418952893
Test

In [11]:
model = ResNet50(input_shape=(IMSIZE,IMSIZE,3), classes = 6)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=20, batch_size=64)
preds = model.evaluate(X_test, Y_test)
print('Loss = ' + str(preds[0]))
print('Test Accuracy = ' + str(preds[1]))

Epoch 1/20
12337/12337 [==============================] - 80s 7ms/step - loss: 1.2351 - acc: 0.5219
Epoch 2/20
12337/12337 [==============================] - 65s 5ms/step - loss: 1.0402 - acc: 0.5597
Epoch 3/20
12337/12337 [==============================] - 65s 5ms/step - loss: 0.7960 - acc: 0.6622
Epoch 4/20
12337/12337 [==============================] - 65s 5ms/step - loss: 0.6683 - acc: 0.7155
Epoch 5/20
12337/12337 [==============================] - 64s 5ms/step - loss: 0.7726 - acc: 0.6950
Epoch 6/20
12337/12337 [==============================] - 65s 5ms/step - loss: 1.6233 - acc: 0.4215
Epoch 7/20
12337/12337 [==============================] - 65s 5ms/step - loss: 1.7098 - acc: 0.3550
Epoch 8/20
12337/12337 [==============================] - 65s 5ms/step - loss: 1.4800 - acc: 0.3853
Epoch 9/20
12337/12337 [==============================] - 65s 5ms/step - loss: 1.2667 - acc: 0.4784
Epoch 10/20
12337/12337 [==============================] - 65s 5ms/step - loss: 1.3149 - acc: 0.4901